In [ ]:
# Chapter 5 데코레이터를 사용한 코드개선

In [ ]:
# function decorator
class ControlledException(Exception):
    """도메인에서 발생하는 일반적인 예외"""


def retry(operation):
    @wraps(operation)
    def wrapped(*args, **kwargs):
        last_raised = None
        RETRIES_LIMIT = 3
        for _ in range(RETRIES_LIMIT):
            try:
                return operation(*args, **kwargs)
            except ControlledException as e:
                logger.info("%s 재시도", operation.__qualname__)
                last_raised = e
            raise last_raised
        return wrapped


@retry
def run_operation(task):
    """실행중 예외가 발생할 것으로 예상되는 특정 작업 실행"""
    return task.run()

In [1]:
# class decorator
from datetime import datetime
from dataclasses import dataclass
from typing import Any


class LoginEventSerializer:
    def __init__(self, event) -> None:
        self.event = event

    def serialize(self) -> dict:
        return {
            "username": self.event.username,
            "password": "** 민감한 정보 삭제 **",
            "ip": self.event.ip,
            "timestamp": self.event.timestamp.strftime("%Y-%m-%d %H:%M"),
        }


@dataclass
class LoginEvent:
    SERIALIZER = LoginEventSerializer

    username: str
    password: str
    ip: str
    timestamp: datetime

    def serialize(self) -> dict:
        return self.SERIALIZER(self).serialize()


################################################################


def hide_field(field) -> str:
    return ("** 민감한 정보 삭제 **",)


def format_time(field_timestampe: datetime) -> str:
    return field_timestampe.strftime("%Y-%m-%d %H:%M")


def show_original(event_field):
    return event_field


class EventSerializer:
    def __init__(self, serialization_fields) -> None:
        self.serialization_fields = serialization_fields

    def serialize(self, event) -> dict:
        return {
            field: transformation(getattr(event, field))
            for field, transformation in self.serialized_fields.items()
        }


class Serialization:
    def __init__(self, **transformations) -> None:
        self.serializer = EventSerializer(transformations)

    def __call__(self, event_class):
        def serialize_method(event_instance):
            return self.serializer.serialize(event_instance)

        event_class.serialize = serialize_method
        return event_class

@Serialization(
    username=show_original,
    password=hide_field,
    ip=show_original,
    timestamp=format_time
)
@dataclass
class LoginEvent:
    username : str
    password : str
    ip : str
    timestamp : datetime

In [ ]:
from typing import Sequence, Optional

_DEFAULT_RETRIES_LIMIT = 3


def with_retry(
    retries_limit: int = _DEFAULT_RETRIES_LIMIT,
    allowed_exceptions: Optional[Sequence[Exception]] = None,
):
    allowed_exceptions = allowed_exceptions or (ControlledException,)

    def retry(operation):
        @wraps(operation)
        def wrapped(*args, **kwargs):
            last_raised = None
            for _ in range(retries_limit):
                try:
                    return operation(*args, **kwargs)
                except allowed_exceptions as e:
                    logger.warnings(
                        "%s 재시도, 원인: %s", operation.__qualname__, e
                    )
                    last_raised = e
                raise last_raised
            return wrapped

    return retry

@with_retry()
def run_operation(task):
    return task.run()

@with_retry(retries_limit=5)
def run_with_custom_retries_limit(task):
    return task.run()

@with_retry(allowed_exceptions=(AttributeError,))
def run_with_custom_exceptions(task):
    return task.run()

@with_retry(
    retries_limit=4,
    allowed_exceptions=(AttributeError,ZeroDivisionError)
)
def run_with_custom_parameters(task):
    return task.run()

In [ ]:
_DEFAULT_RETRIES_LIMIT = 3


class WithRetry:
    def __init__(
        self,
        retries_limit: int = _DEFAULT_RETRIES_LIMIT,
        allowed_exceptions: Optional[Sequence[Exception]] = None,
    ) -> None:
        self.retries_limit = retries_limit
        self.allowed_exceptions = allowed_exceptions or (ControlledException,)

    def __call__(self, operation):
        @wraps(operation)
        def wrapped(*args, **kwargs):
            last_raised = None
            for _ in range(self.retries_limit):
                try:
                    return operation(*args, **kwargs)
                except self.allowed_exceptions as e:
                    logger.warning(
                        "retrying %s due to %s", operation.__qualname__, e
                    )
                    last_raised = e
            raise last_raised

        return wrapped


@WithRetry(retries_limit=5)
def run_with_custom_retries_limit(task):
    return task.run()



In [ ]:
# 기본값과 함께 데코레이터 사용
@retry()
def my_function():
    pass


# 기본값 없이 데코레이터 사용
@retry
def my_function():
    pass

In [ ]:
def trace_decorator(function):
    def wrapped(*arg, **kwargs):
        logger.info("%s 실행", function.__qualname__)
        return function(*arg, **kwargs)

    return wrapped


In [19]:
from functools import wraps


def my_decorator(function):
    """asdf"""

    def wrapped(a, b):
        a += 1
        b += 1
        return function(a, b)

    return wrapped


def my_w_decorator(function):
    """asdf"""

    @wraps(function)
    def wrapped(a, b):
        a += 1
        b += 1
        return function(a, b)

    return wrapped


def my_function(a, b):
    """add a + b"""
    return a + b


@my_decorator
def my_dc_function(a, b):
    """add a + b"""
    return a + b


@my_w_decorator
def my_w_dc_function(a, b):
    """add a + b"""
    return a + b


a, b = 1, 2

print(my_function(a, b))
print(my_dc_function(a, b))
print(my_w_dc_function(a, b))
print("==" * 30)
print(my_dc_function.__doc__)
print(my_w_dc_function.__doc__)

3
5
5
None
add a + b


In [23]:
"""Clean Code in Python - Chapter 5: Decorators

Universal Decorators: create decorators that can be applied to several
different objects (e.g. functions, methods), and won't fail.

"""
from functools import wraps
# from log import logger

class DBDriver:
    def __init__(self, dbstring: str) -> None:
        self.dbstring = dbstring

    def execute(self, query: str) -> str:
        return f"query {query} at {self.dbstring}"


def inject_db_driver(function):
    """This decorator converts the parameter by creating a ``DBDriver``
    instance from the database dsn string.
    """

    @wraps(function)
    def wrapped(dbstring):
        return function(DBDriver(dbstring))

    return wrapped


@inject_db_driver
def run_query(driver):
    return driver.execute("test_function")


class DataHandler:
    """The decorator will not work for methods as it is defined."""

    @inject_db_driver
    def run_query(self, driver):
        return driver.execute(self.__class__.__name__)

In [ ]:
from functools import wraps
from types import MethodType


class DBDriver:
    def __init__(self, dbstring: str) -> None:
        self.dbstring = dbstring

    def execute(self, query: str) -> str:
        return f"query {query} at {self.dbstring}"


class inject_db_driver:
    """Convert a string to a DBDriver instance and pass this to the wrapped
    function.
    """
    # 뭔지는 모르겠는데 함수면 get 작동안하고, 메소드면 get 작동
    def __init__(self, function) -> None:
        self.function = function
        wraps(self.function)(self)

    def __call__(self, dbstring):
        return self.function(DBDriver(dbstring))

    def __get__(self, instance, owner):
        if instance is None:
            return self
        return self.__class__(MethodType(self.function, instance))


@inject_db_driver
def run_query(driver):
    return driver.execute("test_function_2")


class DataHandler:
    @inject_db_driver
    def run_query(self, driver):
        return driver.execute("test_method_2")

In [26]:
def a(b):
    return b